In [371]:
import pandas as pd
import numpy as np
import cvxpy
from numpy.linalg import inv

In [492]:
#Synthetic Problem Data for Original Testing
m = 10000
n = 200
np.random.seed(1)
X = np.random.randn(m, n)
y = np.random.randn(m)

In [430]:
#Parkinsons data to compare against homework
park = pd.read_csv("/Users/johnnicholas/desktop/ML/project/code/parkinsons_updrs1.csv")
cg = pd.read_csv("/Users/johnnicholas/desktop/ML/project/code/CoralGablesHouses.csv")

In [356]:
park_y=(park.iloc[:,16])
park_X=(park.iloc[:,0:16])

In [578]:
cg_y=(cg.iloc[:,1])
cg_y
cg_X=(cg.iloc[:,2:10])
cg_X

,Age,Living,Lot,Tax,Bathrooms,Bedrooms,Garage,Story
0,62,1035,5500,2905,1,2,1,1
1,55,1038,7062,2464,3,3,0,1
2,56,1071,6900,2263,1,2,1,1
3,55,1076,6893,2415,1,2,1,1
4,56,1083,6400,3298,2,3,0,1
...,...,...,...,...,...,...,...,...
531,63,4920,21750,9881,5,4,2,1
532,8,4933,11701,19428,6,5,2,2
533,64,4962,31150,17635,5,5,2,2
534,33,4972,25055,27814,4,5,2,1


In [522]:
from sklearn import preprocessing
# Get column names first
names = cg_X.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(cg_X)
scaled_df = pd.DataFrame(scaled_df, columns=names)
cg_X = scaled_df
cg_X


,Age,Living,Lot,Tax,Bathrooms,Bedrooms,Garage,Story
0,0.666039,-1.544525,-0.876675,-0.767370,-1.537354,-1.559826,-0.269220,-0.542941
1,0.277214,-1.541168,-0.574640,-0.877084,0.214021,-0.423866,-1.605351,-0.542941
2,0.332761,-1.504246,-0.605965,-0.927089,-1.537354,-1.559826,-0.269220,-0.542941
3,0.277214,-1.498652,-0.607319,-0.889274,-1.537354,-1.559826,-0.269220,-0.542941
4,0.332761,-1.490820,-0.702647,-0.669597,-0.661667,-0.423866,-1.605351,-0.542941
...,...,...,...,...,...,...,...,...
531,0.721586,2.802244,2.265487,0.968155,1.965395,0.712095,1.066910,-0.542941
532,-2.333469,2.816789,0.322374,3.343305,2.841082,1.848055,1.066910,1.803966
533,0.777132,2.849236,4.083107,2.897234,1.965395,1.848055,1.066910,1.803966
534,-0.944808,2.860425,2.904555,5.429617,1.089708,1.848055,1.066910,-0.542941


In [579]:
def CorrPairs(X,p):
    X = pd.DataFrame(X)
    corr = X.corr()
    z=np.array([0,0])
    for i in range(len(corr)):
        for j in range(i,len(corr)):
            if abs(corr.iloc[i,j])>= 0.6 and i!=j:
                q = np.array([i,j])
                z = np.vstack((z,q))
    #if z == [0,0]:
        #return;
    #else:
    z = z[1:len(z),:]
    return z;

In [595]:
z = CorrPairs(cg_X,0.6)
print(z)
for i in range(len(z)):
    print(z[i][0] + z[i][1])

[[1 2]
 [1 3]
 [1 4]
 [1 5]
 [2 3]
 [3 4]
 [4 5]]
3
4
5
6
5
7
9


In [657]:
def HolisticReg(X,y,p,k,c):
    #Convert to numpy array
    X = np.asarray(X)
    y = np.asarray(y)
    #VARIABLES
    m = X.shape[0]
    n = X.shape[1]
    B = cp.Variable(shape = n)
    s = cp.Variable(shape = n, boolean=True)
    h = cp.Variable(shape = n, nonneg=True)
    
    z = CorrPairs(X,c)
    
    #OBJECTIVE
    objective = cp.Minimize(cp.sum_squares(y-X*B)+p*(sum(h))
    cons = [sum(s) <= k]
    #CONSTRAINTS
    
    for i in range(n):
        #bigM
        cons.append(5000000*s[i] >= cp.norm(B[i],1))
        #cons.append(5000000*s[i] >= B[i])
        #cons.append(-5000000*s[i] <= B[i])
        #abs_value
        #cons.append(h[i]>=cp.norm(B[i],1))
        #cons.append(h[i]>=-B[i])
        #pairwise multi-collinearity
    for i in range(len(z)):
        cons.append(s[z[i][0]] + s[z[i][1]] <=1)
        #group-sparsity (For Parkinsons Data Only)
    #g = np.array([0,1])
    #g = np.vstack((g,np.array([0,2])))
    #g = np.vstack((g,np.array([0,3])))
    #g = np.vstack((g,np.array([0,4])))
    #g = np.vstack((g,np.array([5,6])))
    #g = np.vstack((g,np.array([6,7])))
    #g = np.vstack((g,np.array([7,8])))
    #g = np.vstack((g,np.array([8,9])))
    #g = np.vstack((g,np.array([9,10])))
    #for i in range(len(g)):
        #cons.append(s[g[i][0]] == s[g[i][1]])
    
        
        
    #INITIALIZE PROBLEM
    prob = cp.Problem(objective, cons)
    
    #SOLVE
    result = prob.solve()

    #GET VALUES
    print(prob.status)
    print(B.value)
    print(s.value)

    return B.value;

SyntaxError: invalid syntax (<ipython-input-657-abc13480d2c9>, line 16)

In [653]:
B = HolisticReg(cg_X,cg_y,0,16,0.6)

IndexError: Index 4 is out of bounds for axis 0 with size 4.

I think my main issue right now is that the objective function is operating differently (https://www.cvxpy.org/examples/basic/least_squares.html). Need to comapre to Michael's answers to see if I did hw wrong or if this is actually wrong... 

New issue. Solver wont solve unless there is a regularizer term that is active that seems arbitrary. Weird. OLS should be able to solve fine.

In [593]:
B = np.array(B)
cg_X = np.array(cg_X)
cg_y = np.array(cg_y)
y_hat = np.matmul(cg_X,B)
mean_squared_error(cg_y,y_hat)

In [584]:
y_hat = np.matmul(cg_X,B)

In [576]:
#j = np.matrix.mean(np.square(np.subtract(y_hat,cg_y)))
j = np.subtract(y_hat,cg_y)
j = np.square(j)
j.shape

(536,)

In [594]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(cg_y,y_hat)

21157161610.81046

In [633]:
def HolisticReg(X,y,p,k,c):
    def loss_fn(X, Y, beta):
        return cp.norm2(cp.matmul(X, beta) - Y)**2

    def regularizer(beta):
        return cp.norm1(beta)

    def objective_fn(X, Y, beta, lambd):
        return loss_fn(X, Y, beta) + lambd * regularizer(beta)
 
    #Convert to numpy array
    X = np.asarray(X)
    y = np.asarray(y)
    #VARIABLES
    m = X.shape[0]
    n = X.shape[1]
    B = cp.Variable(n)
    s = cp.Variable(n, boolean=True)
    h = cp.Variable(n)
    z = CorrPairs(X,c)
    
    #OBJECTIVE
    objecive = cp.Minimize(objective_fn(X, y, B, p))
    
    #CONSTRAINTS
    cons = [sum(s)<=k]
    for i in range(n):
        #bigM
        cons.append(5000000*s[i] >=  B[i])
        cons.append(5000000*s[i] >= -B[i])
        #abs_value
        cons.append(h[i]>=B[i])
        cons.append(h[i]>=-B[i])
        cons.append(h[i]>=0)
        #pairwise multi-collinearity
    for i in range(len(z)):
        cons.append(s[z[i][0]] + s[z[i][1]] <=1)
        #group-sparsity (For Parkinsons Data Only)
    #g = np.array([0,1])
    #g = np.vstack((g,np.array([0,2])))
    #g = np.vstack((g,np.array([0,3])))
    #g = np.vstack((g,np.array([0,4])))
    #g = np.vstack((g,np.array([5,6])))
    #g = np.vstack((g,np.array([6,7])))
    #g = np.vstack((g,np.array([7,8])))
    #g = np.vstack((g,np.array([8,9])))
    #g = np.vstack((g,np.array([9,10])))
    #for i in range(len(g)):
        #cons.append(s[g[i][0]] == s[g[i][1]])
    
        
        
    #INITIALIZE PROBLEM
    prob = cp.Problem(objective, cons)
    
    #SOLVE
    result = prob.solve()

    #GET VALUES
    print(prob.status)
    print(B.value)
    print(s.value)

    return B.value;

In [635]:
B = HolisticReg(cg_X,cg_y,1,16,0.6)

infeasible_inaccurate
None
None


In [614]:
B = [B[0],0,B[2],0,0,B[5],B[6],0]
B

[-648.8652824281094,
 0,
 6.72514220345664,
 0,
 0,
 -8727.039159901573,
 1312.5863728667266,
 0]

In [619]:
B = np.array(B)
cg_X = np.array(cg_X)
cg_y = np.array(cg_y)
y_hat = np.matmul(cg_X,B)
mean_squared_error(cg_y,y_hat)

21121917646.12026